In [1]:
import math
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
import numpy as np
import pandas as pd

In [2]:
def calculate_sentence_log_prob(model, tokenizer, sentence, device):
    inputs = tokenizer(sentence, return_tensors="pt").to(device)
    
    with torch.no_grad():
        outputs = model(**inputs, labels=inputs["input_ids"])
    
    loss = outputs.loss.item() # -mean(log_prob)
    num_tokens = inputs["input_ids"].shape[1]
    
    sentence_log_prob = -loss * (num_tokens - 1)

    perplexity = math.exp(loss)
    
    return sentence_log_prob, perplexity

In [4]:
model_id = "deepseek-ai/DeepSeek-R1-Distill-Llama-8B"
device = "cuda" if torch.cuda.is_available() else "cpu"

print(f"Loading {model_id}...")
tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    model_id, 
    device_map="auto", 
    torch_dtype=torch.float16, 
    trust_remote_code=True
)

Loading deepseek-ai/DeepSeek-R1-Distill-Llama-8B...


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/826 [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'frozen' attribute with value True was provided to the `Field()` function, which has no effect in the context it was used. 'frozen' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` 

...

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-000002.safetensors:   0%|          | 0.00/7.39G [00:00<?, ?B/s]

model-00001-of-000002.safetensors:   0%|          | 0.00/8.67G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/181 [00:00<?, ?B/s]

### Testing on Animate Subject Passive

In [5]:
data1 = pd.read_csv("/kaggle/input/datass/animate_subject_passive.csv")

for index, row in data1.iterrows():
  s1 = row["rows__row__sentence_good"]         
  s2 = row["rows__row__sentence_bad"]   

  log_prob_s1, ppl_s1 = calculate_sentence_log_prob(model, tokenizer, s1, device)
  log_prob_s2, ppl_s2 = calculate_sentence_log_prob(model, tokenizer, s2, device)

  data1.loc[index, "log(P(S1))"] = log_prob_s1
  data1.loc[index, "log(P(S2))"] = log_prob_s2

  if log_prob_s1 > log_prob_s2:
      data1.loc[index, "log(P(S1)) > log(P(S2))"] = 1
  else:
      data1.loc[index, "log(P(S1)) > log(P(S2))"] = 0
  data1.loc[index, "Confidence"] = abs(log_prob_s1 - log_prob_s2)
  data1.loc[index, "Ppl(S1)"] = ppl_s1
  data1.loc[index, "Ppl(S2)"] = ppl_s2 

In [6]:
data1

,rows__row__sentence_good,rows__row__sentence_bad,log(P(S1)),log(P(S2)),log(P(S1)) > log(P(S2)),Confidence,Ppl(S1),Ppl(S2)
0,Amanda was respected by some waitresses.,Amanda was respected by some picture.,-63.640267,-65.372803,1.0,1.732536,1177.490510,3539.003049
1,Some lake was passed by some cashiers.,Some lake was passed by some phenomena.,-61.946257,-58.601727,0.0,3.344530,975.468047,1518.101495
2,Lisa was kissed by the boys.,Lisa was kissed by the blouses.,-43.519531,-56.455135,1.0,12.935605,501.235387,1160.831593
3,Amanda isn't respected by the children.,Amanda isn't respected by the cups.,-49.156746,-59.995282,1.0,10.838536,235.535267,785.360198
4,The glove was noticed by some woman.,The glove was noticed by some mouse.,-54.398510,-58.153389,1.0,3.754879,897.680081,1435.363810
...,...,...,...,...,...,...,...,...
95,All public parks were explored by some teenager.,All public parks were explored by some screen.,-61.755704,-65.129661,1.0,3.373957,955.032031,1389.402405
96,This cake is seen by the pedestrians.,This cake is seen by the parentheses.,-53.455513,-56.046677,1.0,2.591164,797.864772,1103.050254
97,Every person wasn't cared for by the offspring.,Every person wasn't cared for by the fungus.,-55.524359,-56.796637,1.0,1.272278,257.864918,292.850915
98,Edward wasn't insulted by some alumni.,Edward wasn't insulted by some oxen.,-60.123831,-62.262559,1.0,2.138728,796.658133,505.857947


In [7]:
data1.to_csv("/kaggle/working/animate_subject_passive_output.csv")

### Testing on Animate Subject Trans

In [8]:
data2 = pd.read_csv("/kaggle/input/datass/animate_subject_trans.csv")

for index, row in data2.iterrows():
  s1 = row["rows__row__sentence_good"]         
  s2 = row["rows__row__sentence_bad"]   

  log_prob_s1, ppl_s1 = calculate_sentence_log_prob(model, tokenizer, s1, device)
  log_prob_s2, ppl_s2 = calculate_sentence_log_prob(model, tokenizer, s2, device)

  data2.loc[index, "log(P(S1))"] = log_prob_s1
  data2.loc[index, "log(P(S2))"] = log_prob_s2

  if log_prob_s1 > log_prob_s2:
      data2.loc[index, "log(P(S1)) > log(P(S2))"] = 1
  else:
      data2.loc[index, "log(P(S1)) > log(P(S2))"] = 0
  data2.loc[index, "Confidence"] = abs(log_prob_s1 - log_prob_s2)
  data2.loc[index, "Ppl(S1)"] = ppl_s1
  data2.loc[index, "Ppl(S2)"] = ppl_s2

In [9]:
data2

,rows__row__sentence_good,rows__row__sentence_bad,log(P(S1)),log(P(S2)),log(P(S1)) > log(P(S2)),Confidence,Ppl(S1),Ppl(S2)
0,Tina revealed Margaret.,The horse revealed Margaret.,-49.785070,-53.840303,1.0,4.055233,21099.699631,47479.849749
1,Danielle visited Irene.,The eye visited Irene.,-43.204498,-48.377743,1.0,5.173244,5658.418178,15923.456513
2,Paul runs around the art galleries.,The river runs around the art galleries.,-48.144957,-49.715687,1.0,1.570730,970.538439,499.926381
3,Most banks have praised Raymond.,The jackets have praised Raymond.,-60.338293,-57.053970,0.0,3.284323,23304.043826,13480.440422
4,Every doctor was selling some restaurants.,A cup was selling some restaurants.,-56.146926,-62.817699,1.0,6.670774,3044.187818,7894.780010
...,...,...,...,...,...,...,...,...
95,Lawrence fled from Brad.,Literature fled from Brad.,-51.921186,-51.907722,0.0,0.013464,5730.345328,5717.500875
96,These dancers should investigate Caroline.,Glaciers should investigate Caroline.,-53.478722,-54.581169,1.0,1.102448,7428.804847,2434.045278
97,Mitchell can't complain about Peter.,This window can't complain about Peter.,-47.717018,-58.236759,1.0,10.519741,389.407855,1450.400346
98,Every doctor wasn't describing a river.,This story wasn't describing a river.,-55.504440,-49.749718,0.0,5.754723,1030.763469,502.057530


In [10]:
data2.to_csv("/kaggle/working/animate_subject_trans_output.csv")

### Testing on Self Curated Data

In [11]:
data3 = pd.read_csv("/kaggle/input/datass/Self_curated_data.csv")

for index, row in data3.iterrows():
  s1 = row["rows__row__sentence_good"]         
  s2 = row["rows__row__sentence_bad"]   

  log_prob_s1, ppl_s1 = calculate_sentence_log_prob(model, tokenizer, s1, device)
  log_prob_s2, ppl_s2 = calculate_sentence_log_prob(model, tokenizer, s2, device)

  data3.loc[index, "log(P(S1))"] = log_prob_s1
  data3.loc[index, "log(P(S2))"] = log_prob_s2

  if log_prob_s1 > log_prob_s2:
      data3.loc[index, "log(P(S1)) > log(P(S2))"] = 1
  else:
      data3.loc[index, "log(P(S1)) > log(P(S2))"] = 0
  data3.loc[index, "Confidence"] = abs(log_prob_s1 - log_prob_s2)
  data3.loc[index, "Ppl(S1)"] = ppl_s1
  data3.loc[index, "Ppl(S2)"] = ppl_s2  

In [12]:
data3

,rows__row__sentence_good,rows__row__sentence_bad,log(P(S1)),log(P(S2)),log(P(S1)) > log(P(S2)),Confidence,Ppl(S1),Ppl(S2)
0,I eat poha with tea,I eat plate with tea,-54.915526,-45.094457,0.0,9.821069,2553.129474,8257.617070
1,I ordered a cycle,I ordered a wall,-33.135154,-26.548517,0.0,6.586637,755.236428,762.901790
2,I gave him water,I gave him air,-28.301001,-31.226782,1.0,2.925781,1182.339225,2456.997761
3,I eat a tablet,I eat an ipad,-42.002602,-42.645979,1.0,0.643377,4449.381269,5060.374571
4,I and he drank coffee together,I and he drank coffee mug together,-48.818053,-58.192086,1.0,9.374033,1068.496294,4077.156851
5,He has 10 lakh ruppes in his bank,He has 10 lakh cars in his bank,-54.772013,-59.309349,1.0,4.537336,95.998675,376.506347
6,He has calculator in his phone,He has car in his phone,-44.801766,-48.160962,1.0,3.359196,601.996928,972.760050


In [13]:
data3.to_csv("/kaggle/working/Self_curated_data_output.csv")

### Testing on Reference Tracking Data

In [14]:
data4 = pd.read_csv("/kaggle/input/datass/Reference_tracking_data.csv")

for index, row in data4.iterrows():
  s1 = row["rows__row__sentence_good"]         
  s2 = row["rows__row__sentence_bad"]   

  log_prob_s1, ppl_s1 = calculate_sentence_log_prob(model, tokenizer, s1, device)
  log_prob_s2, ppl_s2 = calculate_sentence_log_prob(model, tokenizer, s2, device)

  data4.loc[index, "log(P(S1))"] = log_prob_s1
  data4.loc[index, "log(P(S2))"] = log_prob_s2

  if log_prob_s1 > log_prob_s2:
      data4.loc[index, "log(P(S1)) > log(P(S2))"] = 1
  else:
      data4.loc[index, "log(P(S1)) > log(P(S2))"] = 0
  data4.loc[index, "Confidence"] = abs(log_prob_s1 - log_prob_s2)
  data4.loc[index, "Ppl(S1)"] = ppl_s1
  data4.loc[index, "Ppl(S2)"] = ppl_s2  

In [15]:
data4

,rows__row__sentence_good,rows__row__sentence_bad,log(P(S1)),log(P(S2)),log(P(S1)) > log(P(S2)),Confidence,Ppl(S1),Ppl(S2)
0,Riya gave Anu a gift because she had a birthday.,Riya gave Anu a gift because she had a basket.,-68.728355,-76.139287,1.0,7.410932,197.709063,349.627488
1,Arjun called Kabir because he needed help with...,Arjun called Kabir because the project needed ...,-56.546106,-79.956256,1.0,23.410150,77.455258,468.942999
2,Meera hugged Sanya because she was crying.,Meera hugged Sanya because the hallway was cry...,-61.621051,-81.203316,1.0,19.582265,474.425730,1606.992431
3,The dog barked at the stranger because he ente...,The dog barked at the stranger because the gat...,-52.892193,-62.098664,1.0,9.206471,58.476807,84.404345
4,Amit cleaned the table because it had food sta...,Amit cleaned the table because the food stains...,-61.799952,-80.468302,1.0,18.668351,275.387009,487.782400
5,Tina put the book in the bag because it was to...,Tina put the book in the bag because the bag w...,-56.861504,-63.680724,1.0,6.819221,34.947397,42.348159
6,Rohan thanked Neeraj because he fixed the comp...,Rohan thanked Neeraj because the computer fixe...,-64.557798,-75.908930,1.0,11.351131,216.982462,558.773471
7,Priya covered the pot because it was boiling o...,Priya covered the pot because the boiling wate...,-59.095060,-76.200396,1.0,17.105336,215.352929,351.274837
8,The car hit the pole because it slipped on the...,The car hit the pole because the pole slipped ...,-53.015941,-58.526273,1.0,5.510332,59.036112,65.395147
9,Anita washed the spoon because it was dirty.,Anita washed the spoon because she was dirty.,-54.464726,-61.228609,1.0,6.763883,231.938591,456.167884


In [16]:
data4.to_csv("/kaggle/working/Reference_tracking_data_output.csv")